<a href="https://colab.research.google.com/github/karl-gardner/nanofilm_repo/blob/main/regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. Setup

In [1]:
!git clone https://github.com/karl-gardner/nanofilm_repo  # clone repo
%cd /content/nanofilm_repo

import pandas as pd
import numpy as np
from scipy import optimize
import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib.patches as mpatches
import plotly.graph_objects as go
from google.colab import files
from IPython.display import clear_output
import funcs
clear_output()

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 1.1 Load 3 Dimensional Data from Google Drive

In [3]:
df = pd.read_excel("/content/drive/MyDrive/nanofilm_data/data_files/3d_data.xlsx")
pd.set_option('display.max_columns', None)
data = df.drop(columns=["Unnamed: 5", "Unnamed: 11","Unnamed: 17","Unnamed: 18","Unnamed: 23"]).drop(labels=[0,1,13,24,35,36,46,47,48])
data.head(3)

,Sample 1 (320nm),Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Sample 2 (320nm),Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Sample 3 (320nm),Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Sample 4 (130nm),Unnamed: 20,Unnamed: 21,Unnamed: 22,Sample 5 (220nm),Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28
2,0.058824,0.4,2.114079,2.253873,2.194415,0.058824,0.4,2.145168,1.902623,2.283524,0.058824,0.4,1.932265,1.965487,2.189445,0.058824,0.4,2.239623,2.14324,0.058824,0.4,2.496659,2.259573,2.241109
3,0.117647,0.4,1.256418,1.260329,1.248072,0.117647,0.4,1.42673,1.353488,1.384272,0.117647,0.4,1.247496,1.332744,1.289984,0.117647,0.4,1.510007,1.472898,0.117647,0.4,1.376684,1.40692,1.425315
4,0.176471,0.4,1.146707,1.160197,1.153693,0.176471,0.4,1.324944,1.250044,1.262266,0.176471,0.4,1.149149,1.160333,1.168498,0.176471,0.4,1.15762,1.15013,0.176471,0.4,1.256761,1.277315,1.267016


# 1.2 Define Regression Model and Plot with 3D Model: (Figure XX)

In [4]:
# Choose samples to plot

samples = "s12345"

temp,hum,thick,sizes = funcs.compile_data(data,samples)

xdata = np.stack((temp, hum))
X,Y = np.meshgrid(np.arange(np.amin(temp)-0.05, np.amax(temp)+0.05, (np.amax(temp)-np.amin(temp))/100), 
                  np.arange(np.amin(hum)-0.01, np.amax(hum)+0.03, (np.amax(hum)-np.amin(hum))/100))

def func(x, a, b, c, d):
  return a*np.exp(-1*b*x[0,:] + c*x[1,:]) + d

popt, pcov = optimize.curve_fit(func, xdata, thick)
Z = popt[0]*np.exp(-1*popt[1]*X+popt[2]*Y) + popt[3]

# # This doesn't work for some reason if I do the fig = go.Figure(data=data, layout=layout) way. This must be a bug in plotly
# layout = go.Layout(
#     margin=dict(l=0, r=0, t=0, b=0),
#     width=800*3, height=700*3,
#    yaxis = {"title":{"text":'VALUE'}}
# )

z_offset = (0.05, 0.1)
fig = funcs.make_graph(X,Y,Z,temp,hum,thick,sizes,z_offset,samples)
fig.update_layout(scene_aspectmode="cube",
                  scene =dict(xaxis_title="Norm Temp (Tₙ)",
                              yaxis_title="Normalized RH(RHₙ)",
                              zaxis_title="Normalized Thickness (dₙ)",
                              xaxis = dict(nticks=4, range=[-.1,1.05]),
                              zaxis = dict(nticks=4, range=[np.amin(thick)-z_offset[0],np.amax(thick)+z_offset[1]])),
                  margin=dict(l=0, r=0, t=0, b=0),
                  font = {"size": 17},
                  width=800,
                  height=700)

config = {
  'toImageButtonOptions': {
    'format': 'png', # one of png, svg, jpeg, webp
    'filename': '3d_model',
    'height': 700,
    'width': 800,
    'scale': 5 # Multiply title/legend/axis/canvas sizes by this factor
  }
}
fig.show(config=config)

ssr = np.sum(np.square(thick-func(xdata, *popt)))
sst = np.sum(np.square(thick - np.mean(thick)))
rsquare = 1-ssr/sst

print(f"\nResults for Samples: {samples}")
print(f"\nt = {round(popt[0],2)}e^(-{round(popt[1],2)}*T+{round(popt[2],2)}*RH)+{round(popt[3],2)}")
print(f"\nR^2 = {rsquare} =  {round(rsquare,2)}")
print("\nSSR = ", round(ssr,2))


Results for Samples: s12345

t = 0.37e^(-14.43*T+4.76*RH)+1.02

R^2 = 0.8925141212696655 =  0.89

SSR =  5.97
